# On the Objective Functions
### 1. Context

Given:
\begin{align}
Z &= \begin{bmatrix}
z_{1, 1} & z_{1, 2} & \dots & z_{1, N} \\
z_{2, 1} & z_{2, 2} & \dots & z_{2, N} \\
z_{3, 1} & z_{3, 2} & \dots & z_{3, N} \\
\vdots & \vdots & \ddots & \vdots\\
z_{T, 1} & z_{t, 2} & \dots & z_{T, N} \\
\end{bmatrix}_{T \times N, T >> N} \\
\Sigma & \equiv Z^{T}Z\\
\Sigma &= V\Lambda V^{T} \\
V^{T} V &= I \\
\Lambda &= \begin{bmatrix}
\lambda_{1} & 0 & \dots & 0 \\
0 & \lambda_{2} & \dots & 0 \\
\vdots & \vdots & \ddots & \vdots\\
0 & 0 & \dots & \lambda_{N} \\
\end{bmatrix}_{N \times N} \\
V &= \begin{bmatrix}
v_{1, 1} & v_{1, 2} & \dots & v_{1, N} \\
v_{2, 1} & v_{2, 2} & \dots & v_{2, N} \\
\vdots & \vdots & \ddots & \vdots\\
v_{N, 1} & v_{N, 2} & \dots & v_{N, N} \\
\end{bmatrix}_{N \times N}
\end{align}

We want to find the vector $x$ that maximizes following problem:

\begin{align}
\arg \max_{x} \eta(x)
\end{align}

where

\begin{align}
\eta (x) & \equiv \exp \left( -\sum^{N}_{j=1} \theta_{j} \ln{(\theta_{j})} \right)
\end{align}

and

\begin{align}
\theta = diag\left(V^{T} x\right) \Lambda V^{T} x = \begin{bmatrix}
\lambda_{1} \left( \sum^{N}_{i=1} x_{i}v_{i, 1}\right)^{2} \\
\lambda_{2} \left( \sum^{N}_{i=1} x_{i}v_{i, 2}\right)^{2} \\
\vdots \\
\lambda_{N} \left( \sum^{N}_{i=1} x_{i}v_{i, N}\right)^{2}  \\
\end{bmatrix}
\end{align}

When $\theta$ is normalized so that $\sum_{i=1}^{N} \theta = 1$, $\eta(x)$ is equivalent to the exponential of Shannon's entropy, which is maximized the $\forall \theta_{i}=\theta_{j}$, in which case $\eta (x) = N$.



### 2. Data
I am using random mock data here, just to illustrate the point.

In [1]:
import numpy as np
from scipy.stats import random_correlation
import scipy.linalg
from scipy.optimize import minimize
import itertools

seed = 0

N = 4  # Number of columns of Z
np.random.seed(seed)
l = np.random.exponential(size=4)  # Eigenvectors of ZtZ = Sigma
l = l * 4 / l.sum()
l.sort()
l = l[::-1]

# Sigma Matrix
Sigma = random_correlation.rvs(
    eigs=l,
    random_state=np.random.default_rng(seed=seed)
)

# Eigen-decomposition of Sigma
Lambda, V = np.linalg.eig(Sigma)
sort = Lambda.argsort()[::-1]
Lambda = Lambda[sort]
Lambda = np.diag(Lambda)
V = V[:, sort]

print('Sigma Matrix:\n', Sigma)
print('\nLambda Matrix:\n', Lambda)
print('\nV Matrix:\n', V)

Sigma Matrix:
 [[ 1.         -0.11755444 -0.12899818  0.13803969]
 [-0.11755444  1.          0.04437275 -0.1593243 ]
 [-0.12899818  0.04437275  1.         -0.06812091]
 [ 0.13803969 -0.1593243  -0.06812091  1.        ]]

Lambda Matrix:
 [[1.33530476 0.         0.         0.        ]
 [0.         0.98157024 0.         0.        ]
 [0.         0.         0.84617325 0.        ]
 [0.         0.         0.         0.83695175]]

V Matrix:
 [[ 0.54919594 -0.23416764  0.7219862  -0.3496931 ]
 [-0.50220562 -0.49379785  0.47665858  0.52607015]
 [-0.38818193  0.76968629  0.49717214 -0.09857826]
 [ 0.54358819  0.33001852  0.06597268  0.76892604]]


### 3. Set of possible solutions

Given that we know that the problem above achieve its maximum when $\forall \theta_{i}=\theta_{j}$, the problem has a closed form solution (when unconstrained):

\begin{align}
\theta &= \boldsymbol{1}_{N} \\
diag\left(V^{T} x\right) \Lambda V^{T} x  &= \boldsymbol{1}_{N}\\
\Lambda^{\frac{1}{2}}V^{T} x &= \boldsymbol{1}_{N} \\
x^{\star} &\propto V \Lambda^{-\frac{1}{2}} \boldsymbol{1}_{N}
\end{align}

For a given $\Sigma$, there are $2^{N-1}$ solution vectors ($x$) that can yield  $\eta(w)=N$. This is due to the irrelevance of each eigen-vector's sign in the projection. The set of solutions $X_{N \times 2^{N-1}}$ that spans all the possible linear transformations that result in $\eta(w)=N$ is given by:

\begin{align}
X \propto V \Lambda^{-\frac{1}{2}}J
\end{align}

where

\begin{align}
J = \begin{bmatrix}
\mid & \mid &  & \mid \\
j_{1} & j_{2} & \dots & j_{2^{N-1}} \\
\mid & \mid &  & \mid \\
\end{bmatrix}_{N \times 2^{N-1}}
\end{align}

Where each $j_{i}$ is a vector with entries 1 and -1 as entries that change the sign of each eigenvector, and together all $j$ span every possible combination of signs.

The set of possible 16 solutions is shown below:

In [2]:
combinations = [np.reshape(np.array(i), (N, 1)) for i in itertools.product([1, -1], repeat=N)]
J = np.concatenate(combinations, axis=1)
X = V @ np.linalg.inv(np.sqrt(Lambda)) @ J
X = np.divide(X, X.sum(axis=0))

print('Set of possible solutions:\n')
for i in range(X.shape[1]):
    print(f'Sol {i + 1}:', X[:, i])


Set of possible solutions:

Sol 1: [0.18918188 0.04723977 0.25763433 0.50594402]
Sol 2: [ 0.91295759 -0.64274167  0.70722586  0.02255822]
Sol 3: [ 2.11268329  1.99422041  0.47178804 -3.57869174]
Sol 4: [ 0.07148115  0.88465389 -0.00359226  0.04745722]
Sol 5: [ 0.42195144  0.4381465  -0.25753692  0.39743898]
Sol 6: [ 2.37920012  0.0088052  -0.58833058 -0.79967474]
Sol 7: [ 0.38283854 -0.10142072  1.48014358 -0.7615614 ]
Sol 8: [-0.10161337  0.33852487  0.50825721  0.25483129]
Sol 9: [-0.10161337  0.33852487  0.50825721  0.25483129]
Sol 10: [ 0.38283854 -0.10142072  1.48014358 -0.7615614 ]
Sol 11: [ 2.37920012  0.0088052  -0.58833058 -0.79967474]
Sol 12: [ 0.42195144  0.4381465  -0.25753692  0.39743898]
Sol 13: [ 0.07148115  0.88465389 -0.00359226  0.04745722]
Sol 14: [ 2.11268329  1.99422041  0.47178804 -3.57869174]
Sol 15: [ 0.91295759 -0.64274167  0.70722586  0.02255822]
Sol 16: [0.18918188 0.04723977 0.25763433 0.50594402]


### 4. Original optimization problem

\begin{align}
\arg \max_{x} \hspace{1em} \exp \left( -\sum^{N}_{j=1} \theta_{j} \ln{(\theta_{j})} \right)
\end{align}

The solution given by the code below is one of the columns in the set of solutions. The function, when evaluated, yields N.

In [3]:
def original_objfunc(x, Lambda, V):
    w = x.reshape(-1, 1)
    theta = np.diag((V.T @ w).flatten()) @ Lambda @ V.T @ w
    theta_norm = np.divide(theta.flatten(), theta.sum())  # Normalize the vector so it adds up to 1
    val = -np.exp(-np.sum(np.multiply(theta_norm, np.log(theta_norm))))
    return val

opti = minimize(
    fun=original_objfunc,
    x0=np.array([1 / N] * N),
    args=(Lambda, V),
    method='SLSQP',
    options={'maxiter': 1E9, 'ftol': 1E-14}
)

print('x: ', opti.x / opti.x.sum())
print('eval', original_objfunc(opti.x, Lambda, V))

x:  [ 2.11268298  1.99422014  0.47178802 -3.57869114]
eval -3.9999999999999973


### 3. Alternative optimization problem

\begin{align}
\arg \min_{x} \hspace{1em} \sum_{j=1}^{N} \left( \theta_{j}  - c\right)^{2}
\end{align}

where $c$ is a given constant.

The solution given by the code below also is one of the columns in the set of solutions. The function, when evaluated, yields N.


In [4]:
def alternative_objfunc(x, Lambda, V, constant):
    w = x.reshape(-1, 1)
    theta = np.diag((V.T @ w).flatten()) @ Lambda @ V.T @ w
    c = constant * np.ones(shape=(len(x), 1))
    val = (theta - c).T @ (theta - c)
    return val.item()

opti = minimize(
    fun=alternative_objfunc,
    x0=np.array([1 / N] * N),
    args=(Lambda, V, 1),
    method='SLSQP',
    options={'maxiter': 1E9, 'ftol': 1E-14}
)

print('x: ', opti.x / opti.x.sum())
print('eval', original_objfunc(opti.x, Lambda, V))

x:  [0.18918188 0.04723977 0.25763433 0.50594402]
eval -3.999999999999999
